## 📚 Importación de librerías necesarias

Importación de la librería Pandas para el DataWranling y también de la librería MongoClient para la conexión a la BD almacenada en MongoDB

In [2]:
import pandas as pd
import pymongo as pm
from pymongo import MongoClient as mongo

print(F"✅ ¡Pandas importado correctamente! Versión instalada = {pd.__version__}")
print(F"✅ ¡MongoDB importado correctamente! Versión instalada = {pm.__version__}")

✅ ¡Pandas importado correctamente! Versión instalada = 2.2.2
✅ ¡MongoDB importado correctamente! Versión instalada = 4.13.2


## 🖥️ Conectar a MongoDB y extracción de datos

Utilización del controlador (driver) para conectarme a MongoDB y extraer los datos en un DataFrame.

In [3]:
# Almacenar la conexión a MongoDB en una variable
cliente = mongo("mongodb://localhost:27017/")

# Seleccionar la base de datos y la colección
db = cliente["calidad_datos"]
coleccion = db["clientes_calidad"]

# Obtener todos los registros dentro de la tabla y guardarlos como una lista de diccionarios
registros = list(coleccion.find())

# Convertir todos los registros a un DataFrame para su tratamiento eliminado el atributo "_id"
df = pd.DataFrame(registros)
df.drop(columns=["_id"], inplace=True)
df.head()

,cliente_id,nombre,email,telefono,fecha_nacimiento,direccion,region,estado,ultima_actualizacion,rut
0,C001,Clauida R.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C002,J0sé López,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C003,Ana,ana.correo.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C004,Pedro M.,NaN,5555abc,NaN,NaN,NaN,NaN,NaN,NaN
4,C005,María Gómez,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## ✅ 1) DIMENSIÓN: Completitud

Revisar que los datos realmente tengan valores, que estén presentes, ver si hay valores perdidos, nulos, etc.

### 🔸 A. Revisión general de nulos

Esto ayuda a tener una vista rápida de la completitud de los datos.

In [4]:
# Ver cuántos nulos hay por columna
df.isna().sum()

cliente_id               0
nombre                  10
email                   25
telefono                27
fecha_nacimiento        28
direccion               28
region                  28
estado                  27
ultima_actualizacion    27
rut                     28
dtype: int64

### 🔸 B. Detectar campos vacíos o en blanco (solo espacios)

Aquí usamos .str.strip() para eliminar espacios en blanco antes de evaluar si el campo está efectivamente vacío.

In [5]:
# Detectar campos vacíos, o en blanco, que sean solo espacio
df[df["nombre"].isna() | (df["nombre"].str.strip() == "")]

,cliente_id,nombre,email,telefono,fecha_nacimiento,direccion,region,estado,ultima_actualizacion,rut
5,C006,,NaN,123456789,NaN,NaN,NaN,NaN,NaN,NaN
6,C007,NaN,NaN,NaN,02/11/1990,NaN,NaN,NaN,NaN,NaN
7,C008,NaN,NaN,NaN,1990-11-02,NaN,NaN,NaN,NaN,NaN
8,C009,NaN,NaN,NaN,NaN,Av. Brasil 120,NaN,NaN,NaN,NaN
9,C010,NaN,NaN,NaN,NaN,Avenida Brasil Nº120,NaN,NaN,NaN,NaN
10,C011,NaN,NaN,NaN,NaN,NaN,RM,NaN,NaN,NaN
11,C012,NaN,NaN,NaN,NaN,NaN,Región Metropolitana,NaN,NaN,NaN
21,C022,NaN,email@correo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,C023,NaN,NaN,9999999999999999999999,NaN,NaN,NaN,NaN,NaN,NaN
23,C024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.345.678-X


In [6]:
df[df["email"].isna() | (df["email"].str.strip() == "")]

,cliente_id,nombre,email,telefono,fecha_nacimiento,direccion,region,estado,ultima_actualizacion,rut
0,C001,Clauida R.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C002,J0sé López,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,C004,Pedro M.,NaN,5555abc,NaN,NaN,NaN,NaN,NaN,NaN
4,C005,María Gómez,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,C006,,NaN,123456789,NaN,NaN,NaN,NaN,NaN,NaN
6,C007,NaN,NaN,NaN,02/11/1990,NaN,NaN,NaN,NaN,NaN
7,C008,NaN,NaN,NaN,1990-11-02,NaN,NaN,NaN,NaN,NaN
8,C009,NaN,NaN,NaN,NaN,Av. Brasil 120,NaN,NaN,NaN,NaN
9,C010,NaN,NaN,NaN,NaN,Avenida Brasil Nº120,NaN,NaN,NaN,NaN
10,C011,NaN,NaN,NaN,NaN,NaN,RM,NaN,NaN,NaN


In [7]:
df[df["telefono"].isna() | (df["telefono"].astype(str).str.strip() == "")]

,cliente_id,nombre,email,telefono,fecha_nacimiento,direccion,region,estado,ultima_actualizacion,rut
0,C001,Clauida R.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C002,J0sé López,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C003,Ana,ana.correo.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,C005,María Gómez,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,C007,NaN,NaN,NaN,02/11/1990,NaN,NaN,NaN,NaN,NaN
7,C008,NaN,NaN,NaN,1990-11-02,NaN,NaN,NaN,NaN,NaN
8,C009,NaN,NaN,NaN,NaN,Av. Brasil 120,NaN,NaN,NaN,NaN
9,C010,NaN,NaN,NaN,NaN,Avenida Brasil Nº120,NaN,NaN,NaN,NaN
10,C011,NaN,NaN,NaN,NaN,NaN,RM,NaN,NaN,NaN
11,C012,NaN,NaN,NaN,NaN,NaN,Región Metropolitana,NaN,NaN,NaN


### 🔸 C. Identificar registros sin ningún atributo útil

Acá se definen columnas críticas y se buscan registros vacíos en ellas.

In [8]:
# Identificar registros completos sin ningún atributo útil
# Definir las columnas críticas que voy a evaluar
columnas_criticas = ["nombre","email","telefono","cliente_id"]

# Buscar registros completamente vacíos en estas columnas
df[df[columnas_criticas].isna().all(axis=1)]

,cliente_id,nombre,email,telefono,fecha_nacimiento,direccion,region,estado,ultima_actualizacion,rut


## ✅ 2) DIMENSIÓN: Validez

Validación de formatos y tipos.

### 🔸 A. Validación de formato de correo electrónico

Una verificación básica podría ser revisar que el email contenga un @ y un .

In [10]:
# Validar el formato del correo electrónico
# Filtrar los registros con email NO NULO
emails_no_nulos = df["email"].dropna()

# Aplicar una condición sobre los emails NO NULOS (válidos)
condicion = ~emails_no_nulos.str.contains("@") | ~emails_no_nulos.str.contains(r"\.")

# Mostrar los registros que tengan formato incorrecto
df.loc[emails_no_nulos[condicion].index]

,cliente_id,nombre,email,telefono,fecha_nacimiento,direccion,region,estado,ultima_actualizacion,rut
2,C003,Ana,ana.correo.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,C022,NaN,email@correo,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 🔸 B. Validar formato de teléfono

Esto permite identificar errores como abc123 o 123.

In [11]:
# Detectar teléfonos que contienen letras o símbolos (deben ser numéricos)
df[df["telefono"].notna() & (~df["telefono"].astype(str).str.isnumeric())]

,cliente_id,nombre,email,telefono,fecha_nacimiento,direccion,region,estado,ultima_actualizacion,rut
3,C004,Pedro M.,NaN,5555abc,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Detectar teléfonos demasiado cortos o largos
df[df["telefono"].notna() & (df["telefono"].astype(str).str.len() < 8)]

,cliente_id,nombre,email,telefono,fecha_nacimiento,direccion,region,estado,ultima_actualizacion,rut
3,C004,Pedro M.,NaN,5555abc,NaN,NaN,NaN,NaN,NaN,NaN


### 🔸 C. Validar formato de fechas (fecha_nacimiento y ultima_actualizacion)

Esto detecta fechas con formatos incorrectos como 02/11/1990 si no siguen un patrón ISO.

In [13]:
# Intentar convertir fecha_nacimiento a datetime
df["fecha_valida"] = pd.to_datetime(df["fecha_nacimiento"], errors="coerce")

# Mostrar registros con fechas inválidas
df[df["fecha_nacimiento"].notna() & df["fecha_valida"].isna()]

,cliente_id,nombre,email,telefono,fecha_nacimiento,direccion,region,estado,ultima_actualizacion,rut,fecha_valida
7,C008,NaN,NaN,NaN,1990-11-02,NaN,NaN,NaN,NaN,NaN,NaT


In [14]:
df["ultima_actualizacion_valida"] = pd.to_datetime(df["ultima_actualizacion"], errors="coerce")

df[df["ultima_actualizacion"].notna() & df["ultima_actualizacion_valida"].isna()]

,cliente_id,nombre,email,telefono,fecha_nacimiento,direccion,region,estado,ultima_actualizacion,rut,fecha_valida,ultima_actualizacion_valida


### 🔸 D. Validar valores permitidos en campo estado

Aquí puedes identificar valores como True, 1, "Activo" (con mayúsculas), etc.

In [15]:
# Mostrar valores únicos para detectar inconsistencias
df["estado"].value_counts(dropna=False)

estado
NaN       27
True       2
Activo     1
Name: count, dtype: int64

In [ ]:
# Ver registros con valores inesperados
valores_validos = ["activo", "inactivo"]
df[~df["estado"].astype(str).str.lower().isin(valores_validos)]

## 🔄 Lista de tareas siguientes para aplicar en Python

### 3) DIMENSIÓN: Consistencia
Evalúa si los datos tienen sentido lógico o representan correctamente lo que dicen:
- Verificar valores distintos que significan lo mismo ("RM" vs "Región Metropolitana")
- Direcciones escritas de distintas formas ("Av." vs "Avenida")

### 4) DIMENSIÓN: Unicidad
Asegura que las claves primarias o datos únicos no estén repetidos:
- Duplicados en cliente_id
- Duplicados en nombre o email (casos con nombres similares o errores tipográficos)

### 5) DIMENSIÓN: Actualidad
Detecta registros que pueden estar desactualizados:
- Revisar ultima_actualizacion con fechas antiguas (por ejemplo, antes de 2018)
- Analizar si hay registros sin cambios en mucho tiempo

### 🛠️ Data Wrangling: Correcciones básicas
Aplica acciones correctivas para facilitar análisis:
- Rellenar campos nulos con textos estándar
- Estandarizar estado a "activo" o "inactivo" (todo en minúsculas)

### 📊 Resumen de hallazgos
Entrega una visión global del estado de la calidad:
- Cuántos registros fueron afectados por cada tipo de error
- Posibles acciones: eliminar, corregir o revisar manualmente

### 💾 Persistencia de los datos en MongoDB
Opcional. Si quieres dar el paso final puedes:
- Exportar el DataFrame corregido a un nuevo archivo .csv o .json
- Incluso volver a insertarlo en MongoDB como versión corregida